# Proyecto del Sprint 9:
## Megaline | Recomendador de Plan (Smart = 0, Ultra = 1)

<a id="indice"></a>
## Índice
- [1. Introducción y objetivo](#intro)
- [2. Protocolo y criterios](#protocolo)
- [3. Carga y validaciones de datos](#carga)
- [4. Partición de datos (train/valid/test)](#particion)
- [5. Modelos baseline (comparación inicial)](#baseline)
- [6. Modelos base supervisados](#supervisados)
- [7. Optimización de hiperparámetros (Random Forest)](#optimizacion)
- [8. Evaluación final en test](#evaluacion)
- [9. Pruebas de cordura](#cordura)
  - [9.1 Pruebas básicas (caja negra)](#cordura-basicas)
  - [9.2 Pruebas avanzadas (diagnóstico profundo)](#cordura-avanzadas)
- [10. Conclusiones](#conclusiones)

<a id="intro"></a>
# 1. Introducción y objetivo
**Introducción:**  
La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.  

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.  

**Objetivo:** construir un clasificador para recomendar **Smart (0)** vs **Ultra (1)** con **accuracy ≥ 0.75** en test bloqueado.  

**Contexto:** migración desde planes heredados maximizando conversión sin inflar reclamos.  

**Importancia:** asignación del plan afecta ingresos y satisfacción.  
Métricas/criterios: accuracy (principal), precision/recall/F1 por clase, matriz de confusión; IC 95% por bootstrap.  

[Volver al Índice](#indice)

<a id="protocolo"></a>
# 2. Protocolo y criterios
**Objetivo:**  
Importar todas las librerías necesarias para análisis, modelado y métricas; fijar parámetros globales de reproducibilidad; y definir funciones auxiliares para centralizar el cálculo y presentación de métricas.  

**Contexto:**  
El proyecto requiere múltiples etapas: partición de datos, entrenamiento de modelos (árboles, regresión logística, random forest), validación con CV, y evaluación final con métricas estándar. Además, la reproducibilidad es fundamental para que los resultados sean consistentes en distintas corridas.  

**Importancia:**  
- **Imports ordenados:** concentra todas las dependencias en un bloque para claridad y trazabilidad.  
- **Semilla global (`RANDOM_STATE`):** garantiza que los splits de datos, el muestreo y el entrenamiento de modelos sean replicables.  
- **Supresión de warnings:** limpia la salida en notebook, enfocándose en lo relevante para el análisis.  
- **Función auxiliar (`resumen`):** estandariza el reporte de métricas, evitando repetir lógica en cada modelo. Esto asegura consistencia y comparabilidad.  

**Métricas/criterios:**  
- **Accuracy:** proporción global de aciertos.  
- **Precision, Recall, F1:** métricas por clase, necesarias para evaluar el balance Smart/Ultra.  
- **Matriz de confusión:** diagnósticos sobre errores específicos entre clases.  
- **Reporte por clase (`classification_report`):** resumen interpretativo que combina las métricas clave.  

[Volver al Índice](#indice)

In [ ]:
# [Imports]: librerías generales
import warnings
from pathlib import Path
import numpy as np, pandas as pd, matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report, balanced_accuracy_score, f1_score, roc_curve, precision_recall_curve, roc_auc_score
from sklearn.inspection import permutation_importance

# [Configuración global]: reproducibilidad y supresión de warnings
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
warnings.filterwarnings("ignore")

# [Función auxiliar]: cálculo de métricas estándar
def resumen(y_true, y_pred, titulo="Resumen", y_train=None, pos_label=1):
    from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_recall_fscore_support, confusion_matrix
    import numpy as np
    import pandas as pd

    acc = accuracy_score(y_true, y_pred)
    bacc = balanced_accuracy_score(y_true, y_pred)
    # orden fijo de clases: 0 y 1
    prec, rec, f1, sup = precision_recall_fscore_support(
        y_true, y_pred, labels=[0, 1], zero_division=0
    )
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])

    # opcional: accuracy “esperada” del dummy que predice la clase mayoritaria de y_train
    acc_esp = None
    if y_train is not None:
        maj_train = np.bincount(np.array(y_train)).argmax()
        acc_esp = float(np.mean(np.array(y_true) == maj_train))

    print(f"\n\n*** {titulo} ***\n")
    print("Matriz de confusión (filas=verdad, cols=pred):\n\n", cm)
    print(
        "\nAccuracy: {:.3f} | Balanced Acc: {:.3f}{}".format(
            acc, bacc,
            f" | Acc_esperada_dummy: {acc_esp:.3f}" if acc_esp is not None else ""
        )
    )
    # Tabla limpia por clase (prec, rec, f1, supp)
    tabla = pd.DataFrame(
        {
            "precision": prec,
            "recall": rec,
            "f1": f1,
            "support": sup.astype(int),
        },
        index=[0, 1],
    )
    print("\nReporte por clase:\n")
    print(tabla.to_string(float_format=lambda x: f"{x:.3f}"))

    return acc, bacc, prec, rec, f1, sup, cm, acc_esp

<a id="carga"></a>
# 3. Carga y validaciones de datos
**Objetivo:**  
Asegurar que el dataset `users_behavior.csv` se encuentra disponible en rutas candidatas, validarlo contra el esquema esperado y comprobar su calidad mínima (columnas, nulos, duplicados y balance de clases).  

**Contexto:**  
Los modelos posteriores dependen de la correcta lectura y consistencia del dataset. Si faltan columnas, hay valores nulos o duplicados, o si la variable objetivo está desbalanceada, el desempeño y la interpretabilidad del modelo se ven comprometidos.  

**Importancia:**  
- **Ruta dinámica:** permite trabajar tanto en entornos locales como en ejecución en servidor (dos rutas candidatas).  
- **Validación de esquema:** garantiza que el CSV incluye las 5 columnas clave definidas por el negocio (`calls, minutes, messages, mb_used, is_ultra`).  
- **Chequeo de nulos y duplicados:** detecta problemas de calidad de datos que podrían sesgar o romper el entrenamiento.  
- **Distribución de clases:** conocer la proporción de Smart (0) vs Ultra (1) desde el inicio ayuda a interpretar métricas y definir la necesidad de técnicas de balance.  

**Métricas/criterios:**  
- **Integridad de columnas:** todas las columnas esperadas presentes, sin faltantes.  
- **Nulos:** idealmente cero por columna; si existen, decidir imputación o descarte.  
- **Duplicados:** deberían ser cero; en caso contrario, limpiar antes de entrenar.  
- **Distribución objetivo (`is_ultra`):**  
  - Conteo absoluto por clase (value_counts).  
  - Proporción relativa (% de cada clase).  
  - Se utilizará para estratificación en los splits y como referencia para interpretar la métrica de accuracy.  

[Volver al Índice](#indice)

In [ ]:
# Definir rutas candidatas para encontrar el CSV en distintos entornos
CANDIDATE_PATHS = [Path('../data/users_behavior.csv'),
                   Path('./data/users_behavior.csv')]

# Seleccionar la primera ruta que exista
DATA_PATH = next((p for p in CANDIDATE_PATHS if p.exists()), None)
assert DATA_PATH is not None, 'No se encontró users_behavior.csv'

# Leer el dataset
data = pd.read_csv(DATA_PATH)

# Validar que todas las columnas esperadas están presentes
expected_cols = ['calls','minutes','messages','mb_used','is_ultra']
missing = set(expected_cols) - set(data.columns)
assert not missing, f'Faltan columnas: {missing}'

# Contar nulos y duplicados
nulls = data.isnull().sum()
dups = data.duplicated().sum()

# Valores numéricos negativos
negs=(data[["calls","minutes","messages","mb_used"]] <= 0).all().all()

# Revisar balance de la variable objetivo
class_counts = data['is_ultra'].value_counts().sort_index()
class_ratio = class_counts / class_counts.sum()

# Resumen de validaciones
print(f'Ruta: {DATA_PATH}\n')
# Validar tipos
print(f"\n{data.info()}")
print("\nValores negativos:", negs)
print('\nNulos por columna:\n', nulls)
print('\nDuplicados:', dups)
print('\nDistribución objetivo:', class_counts.to_dict(), class_ratio.to_dict())

<a id="particion"></a>
# 4. Partición de datos (train/valid/test)
**Objetivo:**  
Dividir el dataset en subconjuntos de entrenamiento, validación y prueba de manera estratificada, preservando la proporción de clases en cada split.  

**Contexto:**  
- El conjunto de entrenamiento (60%) se usa para ajustar los modelos.  
- El conjunto de validación (20%) permite comparar modelos y tunear hiperparámetros sin sesgar los resultados finales.  
- El conjunto de prueba (20%) se mantiene aislado hasta el final para obtener una estimación honesta del desempeño real.  

**Importancia:**  
- **Estratificación:** asegura que la proporción Smart/Ultra se mantenga en todos los splits.  
- **Separación adecuada:** evita fugas de información y sobreajuste.  
- **Control de tamaños:** garantiza suficientes observaciones en cada subconjunto para métricas confiables.  

**Métricas/criterios:**  
- **Tamaño de los splits:** 60% train, 20% valid, 20% test.  
- **Proporción de clase objetivo:** se imprime para verificar que se mantiene la distribución en cada subconjunto.  
- **Reproducibilidad:** el uso de `RANDOM_STATE` asegura consistencia entre corridas.  

[Volver al Índice](#indice)

In [ ]:
features = ['calls','minutes','messages','mb_used']
X = data[features].copy()
y = data['is_ultra'].astype(int).copy()

# Split 1: 60% train, 40% temporal (valid+test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=RANDOM_STATE
)

# Split 2: 20% valid, 20% test (50/50 a partir del 40% temporal)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=RANDOM_STATE
)

# Resumen de proporciones en cada split
for name, s in [
    ("Total 100%", y),
    ("Train 60%", y_train),
    ("Valid 20%", y_valid),
    ("Test 20%", y_test)
]:
    counts = s.value_counts()
    props = s.value_counts(normalize=True).round(3)
    summary = pd.DataFrame({'count': counts, 'proportion': props})
    print(f"\nSet: {name} (n={counts.sum()})\n", summary)

<a id="baseline"></a>
# 5. Modelo base: Dummy y heurística simple

**Objetivo**  
Definir puntos de referencia mínimos contra los cuales evaluar los modelos supervisados.  

**Contexto**  
Antes de entrenar clasificadores más sofisticados, es indispensable contar con *baselines* que indiquen qué tan lejos se está de una predicción trivial o de una simple regla de negocio.  

**Importancia**  
- **DummyClassifier (clase mayoritaria):** ignora todas las variables y siempre predice la clase más frecuente. Representa la *cota inferior* del desempeño esperado.  
- **Heurística con `mb_used`:** regla basada en un único predictor (uso de MB). Fácil de explicar y aplicar en negocio, sirve para comparar si un modelo estadístico realmente aporta valor más allá de una regla operativa sencilla.  

**Métricas y criterios**  
- **Accuracy y Balanced Accuracy:** verifican si los modelos posteriores superan al predictor trivial.  
- **F1-score:** evalúa la capacidad de capturar correctamente la clase positiva (caso de interés en negocio).  
- **Matriz de confusión:** permite identificar los errores más comunes de cada baseline.  
- **Comparación con la proporción mayoritaria:** ningún modelo debería rendir peor que el Dummy; cualquier mejora por encima de él es evidencia de valor añadido.  

[Volver al Índice](#indice)

In [ ]:
# [Dummy]: Modelo base que siempre predice la clase mayoritaria
# Sirve como referencia mínima de desempeño: ningún modelo real debería rendir peor que esto.
dummy = DummyClassifier(strategy='most_frequent', random_state=RANDOM_STATE)
dummy.fit(X_train, y_train)
_ = resumen(y_valid, dummy.predict(X_valid),
            titulo='Modelo Dummy para predecir clase mayoritaria (valid)')

# Extraemos el predictor 'mb_used' de cada partición (train, valid, test)
# Lo convertimos en arrays de numpy para evaluarlo como regla heurística.
mb_train, mb_valid, mb_test = [df['mb_used'].to_numpy() for df in (X_train, X_valid, X_test)]
yv, yt = np.array(y_valid), np.array(y_test)

# Definimos una rejilla de posibles umbrales (percentiles 1–99 de los valores en train)
# Así probamos distintos puntos de corte para convertir 'mb_used' en predicciones binarias.
candidatos = np.quantile(mb_train, np.linspace(0.01, 0.99, 99))

def mejor_umbral(metric):
    """
    Busca el umbral que maximiza una métrica dada (ej. Balanced Accuracy o F1).
    Retorna el umbral óptimo y el score alcanzado en valid.
    """
    scores = [metric(yv, (mb_valid >= t).astype(int)) for t in candidatos]
    i = np.argmax(scores)
    return candidatos[i], scores[i]

# Métricas que nos interesa optimizar
# - BACC: útil cuando las clases están desbalanceadas.
# - F1: balance entre precisión y recall para la clase positiva.
metricas = {
    "BACC": lambda y,p: balanced_accuracy_score(y,p),
    "F1":   lambda y,p: f1_score(y,p,pos_label=1)
}

# Evaluamos cada métrica: mejor umbral, score alcanzado y accuracy asociado
# Guardamos resultados en un diccionario y mostramos resúmenes en valid.
resultados = {}
for nombre, metrica in metricas.items():
    u, sc = mejor_umbral(metrica)
    acc = accuracy_score(yv, (mb_valid >= u).astype(int))
    resultados[nombre] = {"umbral": u, "score": sc, "acc": acc}
    _ = resumen(y_valid, (mb_valid >= u).astype(int),
                titulo=f"Dummy: Heurística optimizada por {nombre} (valid, ACC={acc:.3f})")

# [Selección del Dummy optimizado]
# Criterio: elegimos el modelo con mayor score, siempre que supere cierto mínimo de accuracy (>=0.75).
ganador = max(resultados.items(),
              key=lambda kv: (kv[1]["acc"] >= 0.75, kv[1]["score"]))
criterio, datos = ganador
u_final = datos["umbral"]

# [Evaluación final en TEST]
# Probamos en test el umbral y criterio ganador, para ver su desempeño fuera de valid.
_ = resumen(y_test, (mb_test >= u_final).astype(int),
            titulo=f"Dummy: Heurística elegida ({criterio}), umbral={u_final:.2f}")

<a id="supervisados"></a>
# 6. Modelos base supervisados
**Objetivo:**  
Entrenar y evaluar tres clasificadores supervisados estándar (Regresión Logística, Árbol de Decisión y Random Forest) en configuración básica, obteniendo métricas iniciales de validación.  

**Contexto:**  
Tras los baselines triviales, es necesario comparar modelos de distinta naturaleza:  
- **Regresión Logística:** lineal, interpretable, requiere escalado de features.  
- **Árbol de Decisión:** no lineal, interpretable, robusto a escalas.  
- **Random Forest:** ensamble de árboles, suele mejorar estabilidad y desempeño.  

**Importancia:**  
- **Comparación justa:** usar la misma partición (train/valid) para los tres modelos.  
- **Escalado previo en LogReg:** evita problemas numéricos y asegura convergencia.  
- **Resultados consistentes:** con la función `resumen` se comparan métricas en un mismo formato.  
- **Diccionario `base_scores`:** conserva las exactitudes para futuras referencias.  

**Métricas/criterios:**  
- **Accuracy y Balanced Accuracy:** desempeño general y balanceado entre clases.  
- **Reporte por clase:** diagnóstico detallado en Smart vs Ultra.  
- **Comparación entre modelos:** ver cuál arranca con mejor base en validación.  

[Volver al Índice](#indice)

In [ ]:
# [Pipeline]: Regresión Logística con escalado y balanceo de clases
pipe_lr = Pipeline([
    ('scaler', StandardScaler()),  
    ('model', LogisticRegression(
        random_state=RANDOM_STATE, 
        max_iter=9999,
        class_weight='balanced'))
])

# [Pipeline]: Árbol de Decisión con balanceo
pipe_dt = Pipeline([
    ('model', DecisionTreeClassifier(
        random_state=RANDOM_STATE, 
        class_weight='balanced'))
])

# [Pipeline]: Random Forest con balanceo
pipe_rf = Pipeline([
    ('model', RandomForestClassifier(
        random_state=RANDOM_STATE, 
        n_jobs=-1,
        class_weight='balanced'))
])

# [Entrenamiento y validación]: se evalúan los tres modelos
base_scores_bal = {}
for name, pipe in [
    ('Logistic Regression (balanced)', pipe_lr), 
    ('Decision Tree (balanced)', pipe_dt), 
    ('Random Forest (balanced)', pipe_rf)
]:
    pipe.fit(X_train, y_train)                        
    pred = pipe.predict(X_valid)                      
    acc, *_ = resumen(y_valid, pred,                  
                      titulo=f"Modelo base {name}")
    base_scores_bal[name] = acc


## Resultados obtenidos en validación

### Comparativa de métricas principales

| Modelo              | Accuracy | Balanced Acc | Precisión (0) | Recall (0) | F1 (0) | Precisión (1) | Recall (1) | F1 (1) |
|---------------------|:--------:|:------------:|:-------------:|:----------:|:------:|:-------------:|:----------:|:------:|
| Logistic Regression |  0.641   |    0.632     |     0.79      |   0.66     |  0.72  |     0.44      |   0.61     |  0.51  |
| Decision Tree       |  0.728   |    0.673     |     0.80      |   0.81     |  0.81  |     0.56      |   0.53     |  0.55  |
| Random Forest       |  0.796   |    0.727     |     0.82      |   0.91     |  0.86  |     0.72      |   0.55     |  0.62  |

---

### Detalle por modelo

**Logistic Regression (balanced)**  
- Accuracy = **0.641**  
- Smart (0): precision 0.79, recall 0.66, f1 0.72  
- Ultra (1): precision 0.44, recall 0.61, f1 0.51  
- **Interpretación:** el recall en Ultra mejora notablemente (0.21 → 0.61), aunque cae el accuracy global. Buen trade-off si la prioridad es no perder clientes Ultra.  

---

**Decision Tree (balanced)**  
- Accuracy = **0.728**  
- Smart (0): precision 0.80, recall 0.81, f1 0.81  
- Ultra (1): precision 0.56, recall 0.53, f1 0.55  
- **Interpretación:** mantiene un desempeño equilibrado, con recall en Ultra levemente inferior a Logistic Regression, pero mejor accuracy total.  

---

**Random Forest (balanced)**  
- Accuracy = **0.796**  
- Smart (0): precision 0.82, recall 0.91, f1 0.86  
- Ultra (1): precision 0.72, recall 0.55, f1 0.62  
- **Interpretación:** sigue siendo el más sólido en accuracy y mantiene buen balance; el recall en Ultra sube respecto al modelo sin balanceo.  

---

## Conclusión preliminar

- El balanceo de clases mejora el **recall de Ultra**, sobre todo en Logistic Regression.  
- El costo es una ligera caída en accuracy (esperable en desbalance 70/30).  
- **Random Forest (balanced)** continúa como el mejor modelo integral: combina alto accuracy con mejora en recall para Ultra.  
- **Logistic Regression (balanced)** es útil si la prioridad absoluta es capturar la mayor cantidad de Ultras, aceptando pérdida de exactitud global.  

**Siguiente paso:** realizar búsqueda de hiperparámetros en Random Forest con `class_weight="balanced"` para confirmar si puede mantener este equilibrio en el set de test.


<a id="optimizacion"></a>
# 7. Optimización de hiperparámetros (Random Forest)

**Objetivo:**  
Ajustar los hiperparámetros de Random Forest mediante validación cruzada estratificada, priorizando la **balanced accuracy** como criterio de selección para no subestimar la clase Ultra (minoritaria).  

**Contexto:**  
Entre los modelos base, Random Forest mostró el mejor desempeño. Por ello se concentra el esfuerzo de optimización en este algoritmo, buscando mayor estabilidad y mejor trade-off entre clases.  

**Importancia:**  
- **Validación cruzada (5 folds):** asegura robustez en la evaluación de combinaciones.  
- **Espacio de búsqueda ampliado:** considera número de árboles, profundidad máxima, tamaño mínimo de hojas, criterios de división, selección de features y balanceo de clases.  
- **RandomizedSearchCV:** permite explorar de forma eficiente 30 combinaciones aleatorias sin necesidad de un grid completo.  

**Métricas/criterios:**  
- **Balanced Accuracy:** criterio final de selección, más representativo en escenarios desbalanceados.  
- **Accuracy en validación:** usado como referencia secundaria para confirmar que el modelo no pierde rendimiento global.  
- **Reporte de parámetros óptimos:** asegura trazabilidad y reproducibilidad del modelo elegido.  

[Volver al Índice](#indice)


In [ ]:
# [Validación]: 5 folds estratificados
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# [Espacio de búsqueda Random Forest ampliado]
param_rf = {
    'model__n_estimators': [500, 800, 1000],
    'model__max_depth': [None, 5, 8, 10, 12, 16, 20],
    'model__min_samples_leaf': [1, 2, 5],
    'model__min_samples_split': [2, 5, 10],
    'model__max_features': ['sqrt', 'log2'],
    'model__class_weight': [None, 'balanced']
}

def run_search(scoring_name):
    rs = RandomizedSearchCV(
        pipe_rf,
        param_distributions=param_rf,
        n_iter=30,
        cv=cv,
        scoring=scoring_name,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=0
    )
    rs.fit(X_train, y_train)
    best_rf = rs.best_estimator_
    acc, bacc, prec, rec, f1, sup, cm, _ = resumen(
        y_valid,
        best_rf.predict(X_valid),
        titulo=f"Random Forest optimizado (valid, scoring={scoring_name})"
    )
    return {
        "scoring": scoring_name,
        "params": rs.best_params_,
        "accuracy": acc,
        "balanced_acc": bacc,
        "recall_ultra": rec[1],
        "f1_ultra": f1[1]
    }, best_rf  # <- devuelves también el modelo

# Ejecutar búsquedas con los dos criterios
res_accuracy, _ = run_search("accuracy") # solo el dict
res_bacc, best_rf = run_search("balanced_accuracy") # dict + modelo optimizado

# Mostrar los mejores parámetros de forma legible
print("\n***Mejores parámetros RF (scoring=balanced_accuracy):***\n")
for k, v in res_bacc["params"].items():
    print(f"- {k}: {v}")


## Resultados de la validación (Random Forest optimizado)

### Mejores parámetros seleccionados (scoring = balanced_accuracy)

- `model__n_estimators`: 800  
- `model__min_samples_split`: 2  
- `model__min_samples_leaf`: 5  
- `model__max_features`: log2  
- `model__max_depth`: None  
- `model__class_weight`: balanced  

Estos parámetros se **congelan** para la evaluación final en el set de test.

---

### Comparativa de métricas principales

| Scoring              | Accuracy | Balanced Acc | Precisión (0) | Recall (0) | F1 (0) | Precisión (1) | Recall (1) | F1 (1) |
|----------------------|:--------:|:------------:|:-------------:|:----------:|:------:|:-------------:|:----------:|:------:|
| Accuracy             |  0.815   |    0.742     |     0.825     |   0.930    |  0.875 |     0.779     |   0.553    |  0.647 |
| Balanced Accuracy    |  0.793   |    0.743     |     0.837     |   0.872    |  0.854 |     0.680     |   0.614    |  0.645 |

---

### Detalle por configuración

**Random Forest (scoring=accuracy)**  
- Accuracy = **0.815**  
- Smart (0): precision 0.825, recall 0.930, f1 0.875  
- Ultra (1): precision 0.779, recall 0.553, f1 0.647  
- **Interpretación:** maximiza la exactitud global, pero sigue sacrificando la clase Ultra (recall 0.553).  

---

**Random Forest (scoring=balanced_accuracy)**  
- Accuracy = **0.793**  
- Smart (0): precision 0.837, recall 0.872, f1 0.854  
- Ultra (1): precision 0.680, recall 0.614, f1 0.645  
- **Interpretación:** cede algo de accuracy global (~2.2 puntos), pero gana recall en Ultra (0.553 → 0.614), ofreciendo un modelo más equilibrado entre clases.  

---

## Conclusión preliminar

- **Scoring=accuracy:** prioriza la exactitud global, pero deja escapar demasiados clientes Ultra.  
- **Scoring=balanced_accuracy:** mejora el recall de Ultra y mantiene métricas globales competitivas, alineándose mejor con el objetivo de negocio.  

**Elección final:** se selecciona **Random Forest optimizado con `scoring="balanced_accuracy"`** (parámetros arriba) como modelo definitivo a evaluar en el set de test.  


<a id="evaluacion"></a>
# 8. Evaluación final en test

**Objetivo:**  
Evaluar el desempeño del modelo final (Random Forest optimizado) en el conjunto de prueba independiente, obteniendo métricas estándar y un intervalo de confianza robusto mediante bootstrap con reentrenamiento.  

**Contexto:**  
Una vez seleccionados y optimizados los hiperparámetros en `train+valid`, se reentrena el mejor modelo con todos esos datos para aprovechar la máxima información disponible antes de la evaluación en `test`.  

**Importancia:**  
- **Métricas en test:** entregan una estimación honesta del desempeño real sobre datos nunca vistos.  
- **Bootstrap con reentrenamiento:** captura no solo la variabilidad de muestreo del test, sino también la del entrenamiento, ofreciendo un IC más realista.  
- **Intervalo de confianza (95%):** aporta una medida de incertidumbre sobre la métrica principal, útil en reportes técnicos y ejecutivos.  

**Métricas/criterios:**  
- **Accuracy y Balanced Accuracy en test.**  
- **Reporte de precisión, recall y F1 por clase.**  
- **IC 95% (bootstrap con reentrenamiento):** rango esperado de accuracy.  

[Volver al Índice](#indice)


In [ ]:
# Reentrenar el mejor modelo (Random Forest optimizado) en train+valid
X_tv = pd.concat([X_train, X_valid], axis=0)
y_tv = pd.concat([y_train, y_valid], axis=0)
best_rf.fit(X_tv, y_tv)

# Evaluación estándar en test
pred_test = best_rf.predict(X_test)
print("*** Evaluación Final ***")
acc_std, *_ = resumen(y_test, pred_test, titulo="Modelo RF optimizado (test)")

# Bootstrap IC 95% para accuracy (con reentrenamiento)
B = 300   
n = len(y_tv)
rng = np.random.default_rng(RANDOM_STATE)
boot = []

# Guardar los hiperparámetros óptimos del pipeline completo
best_params = best_rf.get_params()

for _ in range(B):
    # Remuestrear train+valid con reemplazo
    idx = rng.integers(0, n, n)
    Xb, yb = X_tv.iloc[idx], y_tv.iloc[idx]
    
    # Nueva instancia del pipeline con los mismos parámetros
    pipe_b = Pipeline([('model', RandomForestClassifier())])
    pipe_b.set_params(**best_params)
    
    # Reentrenar y evaluar
    pipe_b.fit(Xb, yb)
    pred_b = pipe_b.predict(X_test)
    boot.append(accuracy_score(y_test, pred_b))

low, high = np.percentile(boot, [2.5, 97.5])
print(f"\nIC 95% accuracy (bootstrap con reentrenamiento): [{low:.3f}, {high:.3f}]")


## Resultados de evaluación (Random Forest optimizado)

### Métricas principales

- Accuracy = **0.790**  
- Balanced Accuracy = **0.748**  

### Matriz de confusión (filas = verdad, columnas = predicción)

|              | Pred. 0 | Pred. 1 |
|--------------|---------|---------|
| **Real 0**   |   382   |   64    |
| **Real 1**   |   71    |   126   |

### Desempeño por clase

| Clase | Precisión | Recall | F1   | Soporte |
|-------|-----------|--------|------|---------|
| 0 (Smart) | 0.843     | 0.857  | 0.850 | 446     |
| 1 (Ultra) | 0.663     | 0.640  | 0.651 | 197     |

### Intervalo de confianza

- IC 95% accuracy (bootstrap con reentrenamiento): **[0.764, 0.802]**

---

**Interpretación:**  
El modelo final mantiene un buen balance: precisión alta para la clase mayoritaria (Smart) y un recall aceptable para Ultra (0.640), bastante mejor que baselines. El intervalo de confianza indica que el accuracy real esperado se encuentra estable en torno a 0.79.  

---

### Comparativa con baselines

| Modelo                     | Accuracy | Balanced Acc | Precisión (0) | Recall (0) | F1 (0) | Precisión (1) | Recall (1) | F1 (1) |
|-----------------------------|:--------:|:------------:|:-------------:|:----------:|:------:|:-------------:|:----------:|:------:|
| Dummy (mayoría)            |  0.694   |    0.500     |     0.694     |   1.000    |  0.820 |     0.000     |   0.000    |  0.000 |
| Heurística MB (umbral ópt.)|  0.593   |    0.617     |     0.747     |   0.675    |  0.709 |     0.270     |   0.558    |  0.362 |
| RF optimizado (final)      |  0.790   |    0.748     |     0.843     |   0.857    |  0.850 |     0.663     |   0.640    |  0.651 |

**Conclusión comparativa:**  
- El **Dummy** marca la cota mínima: incapaz de predecir Ultras.  
- El modelo **heurístico** mejora recall en Ultra pero sacrifica accuracy global.  
- El **Random Forest optimizado** domina en todas las métricas clave, logrando un equilibrio robusto entre clases y un desempeño realista en test.

<a id="cordura"></a>
# 9. Pruebas de cordura
**Objetivo:**  
Confirmar que el modelo final (Random Forest optimizado) funciona de manera coherente, que no depende de errores o fugas de información, y que sus predicciones tienen sentido tanto práctico como estadístico.  

**Contexto:**  
Aunque las métricas en `test` ya son buenas, siempre es recomendable realizar pruebas adicionales para descartar trampas involuntarias y para validar la interpretabilidad del modelo. Aquí se combinan **pruebas básicas** (rápidas y lógicas) con **pruebas avanzadas** (más técnicas y visuales).  

[Volver al Índice](#indice)

<a id="cordura-basicas"></a>
## 9.1 Pruebas básicas (caja negra)

1. **Predicción constante**  
   Verificamos que el modelo no esté prediciendo siempre la clase mayoritaria.  
   - Esperado: el modelo predice ambas clases (0=Smart, 1=Ultra).  

2. **Distribución de predicciones vs real**  
   Comparamos la proporción real de clases con la proporción de predicciones.  
   - Esperado: proporciones similares; si hay desviación fuerte, el modelo podría estar sesgado.  

3. **Baseline aleatorio**  
   Simulamos un modelo que asigna etiquetas al azar.  
   - Esperado: accuracy cercano a 0.5 (azar puro). El RF debe superarlo ampliamente.  

4. **Bootstrap de accuracy**  
   Calculamos un IC 95% para accuracy repitiendo muestreo del test set.  
   - Esperado: un intervalo relativamente estrecho alrededor de la métrica observada.  

[Volver al Índice](#indice)

In [ ]:
# 1. ¿El modelo predice solo una clase?
unique_preds, counts = np.unique(pred_test, return_counts=True)
print("Clases predichas en test:", dict(zip(unique_preds, counts)))

# 2. Distribución de predicciones vs distribución real
print("\nDistribución real (test):", y_test.value_counts(normalize=True).to_dict())
print("Distribución predicha (test):",
      pd.Series(pred_test).value_counts(normalize=True).to_dict())

# 3. Baseline aleatorio
rng = np.random.default_rng(RANDOM_STATE)
rand_preds = rng.integers(0, 2, size=len(y_test))
acc_rand = accuracy_score(y_test, rand_preds)
print(f"\nAccuracy baseline aleatorio: {acc_rand:.3f}")

# 4. Intervalo bootstrap ya calculado arriba
print(f"\nIC 95% accuracy (bootstrap, estándar): [{low:.3f}, {high:.3f}]")


<a id="cordura-avanzadas"></a>
## 9.2 Pruebas avanzadas (diagnóstico profundo)

1. **Barajado de etiquetas (sanity check extremo)**  
   Se reentrena el modelo con las etiquetas aleatoriamente barajadas.  
   - **Esperado:** el accuracy debe caer a ~0.5 (azar puro). Si el valor es mayor, puede existir fuga de información.  

2. **Importancia por permutación**  
   Se calcula la contribución real de cada feature en el desempeño del modelo, evaluando cómo cambia la métrica al alterar aleatoriamente sus valores.  
   - **Esperado:** `mb_used` debe aparecer como la variable más relevante, lo cual coincide con la intuición de negocio (a mayor uso de MB, más probabilidad de migrar a Ultra).  

3. **Curvas ROC y PR**  
   Se grafican las curvas ROC (tasa de verdaderos positivos vs tasa de falsos positivos) y PR (precisión vs recall).  
   - **Esperado:**  
     - La curva ROC debe ubicarse por encima de la diagonal aleatoria.  
     - La curva Precision-Recall debe quedar por encima de la línea base (proporción de la clase positiva = Ultra).  

[Volver al Índice](#indice)

In [ ]:
# 1. Repetir varias veces la prueba de etiquetas barajadas
B = 20
accs, baccs, aucs = [], [], []

for b in range(B):
    # Barajar etiquetas
    y_tv_shuf = y_tv.sample(frac=1.0, random_state=RANDOM_STATE + b)
    rf_shuf = RandomForestClassifier(random_state=RANDOM_STATE + b, n_jobs=-1)
    rf_shuf.fit(X_tv, y_tv_shuf)
    
    # Predicciones en test
    pred_shuf = rf_shuf.predict(X_test)
    probs_shuf = rf_shuf.predict_proba(X_test)[:, 1]
    
    # Métricas
    accs.append(accuracy_score(y_test, pred_shuf))
    baccs.append(balanced_accuracy_score(y_test, pred_shuf))
    aucs.append(roc_auc_score(y_test, probs_shuf))

print(f"Promedio accuracy (shuffle): {np.mean(accs):.3f}")
print(f"Promedio balanced acc (shuffle): {np.mean(baccs):.3f}")
print(f"Promedio ROC AUC (shuffle): {np.mean(aucs):.3f}")

# 2. Comparar contra DummyClassifier
dummy = DummyClassifier(strategy='most_frequent', random_state=RANDOM_STATE)
dummy.fit(X_train, y_train)
pred_dummy = dummy.predict(X_test)
probs_dummy = np.full_like(y_test, fill_value=0)  # dummy siempre predice 0

print("\nDummyClassifier:")
print(f"Accuracy: {accuracy_score(y_test, pred_dummy):.3f}")
print(f"Balanced Acc: {balanced_accuracy_score(y_test, pred_dummy):.3f}")
print(f"ROC AUC: {roc_auc_score(y_test, probs_dummy):.3f}")

# 2. Permutation importance
try:
    result = permutation_importance(best_rf, X_test, y_test, n_repeats=20,
                                    random_state=RANDOM_STATE, n_jobs=-1)
    importances = pd.Series(result.importances_mean, index=features).sort_values(ascending=False)
    print('\nImportancias por permutación (test):\n', importances)
    plt.figure()
    importances.sort_values().plot(kind='barh')
    plt.title('Importancia por permutación (test)')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print('Permutation importance no disponible:', str(e))

# 3. Curvas ROC y PR
if hasattr(best_rf, "predict_proba"):
    probs = best_rf.predict_proba(X_test)[:,1]
    fpr, tpr, _ = roc_curve(y_test, probs)
    prec, rec, _ = precision_recall_curve(y_test, probs)
    
    # ROC
    plt.figure()
    plt.plot(fpr, tpr, label="ROC")
    plt.plot([0,1],[0,1], 'k--', label="Random")
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('Curva ROC (diagnóstico)')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # PR
    plt.figure()
    plt.plot(rec, prec, label="PR Curve")
    baseline = y_test.mean()
    plt.hlines(baseline, 0, 1, colors='r', linestyles='--', label="Baseline")
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Curva Precision-Recall (diagnóstico)')
    plt.legend()
    plt.tight_layout()
    plt.show()


**Validación rápida**  
- Pruebas de cordura básicas correctas.  
- Curvas generadas sin error.

<a id="conclusiones"></a>
# 10. Conclusiones

**1. Desempeño global del modelo**  
- El **Random Forest optimizado** alcanzó **Accuracy = 0.790** en test, con **IC 95% ≈ [0.764, 0.802]**, superando claramente los baselines (Dummy ≈ 0.694; Heurística por MB ≈ 0.593 en test).  
- El modelo **predice ambas clases**. La proporción predicha de **Ultra ≈ 29.5%** queda muy cercana a la real (**30.6%**), sin colapso hacia la clase mayoritaria.

**2. Pruebas de cordura básicas**  
- Supera ampliamente al azar (**accuracy aleatorio ≈ 0.462**).  
- La distribución de clases predicha es consistente con la real y no muestra sesgo extremo.  
- El bootstrap confirma estabilidad del desempeño (**IC 95% de accuracy ≈ [0.764, 0.802]**).

**3. Pruebas de cordura avanzadas**  
- **Barajado de etiquetas (20 repeticiones):** el **accuracy** ronda ~0.66 por el desbalance, pero **Balanced Accuracy ≈ 0.514** y **ROC AUC ≈ 0.505** caen a ~0.5, lo que **descarta fuga de información**.  
- **Importancia por permutación:** `mb_used` emerge como el predictor dominante; el resto aporta señal secundaria.  
- **Curvas ROC y PR:** poder discriminativo claro (ROC por encima de la diagonal; PR por encima del baseline de **0.306**).

**4. Implicaciones de negocio**  
- El modelo es **útil para priorizar clientes con alta probabilidad de migrar a Ultra**; mantiene un perfil conservador con **recall en Ultra ≈ 0.640** (vs **0.857** en Smart).  
- Si la prioridad es **no perder Ultras**, puede **ajustarse el umbral** con `predict_proba` para ganar recall aceptando menor precisión. Dado que la proporción predicha ya quedó cercana a la real, el ajuste es una **palanca opcional**, no un requisito.  
- **Consumo de MB** es la variable clave; las acciones comerciales enfocadas en usuarios con altos consumos tendrán mayor impacto.

---

### **Conclusión final**
El Random Forest optimizado es **robusto, consistente y alineado con negocio**: supera baselines, no muestra fuga de información y captura la señal más relevante (`mb_used`).  
El siguiente paso es **definir el umbral operativo** según la tolerancia a falsos negativos en Ultra y trazar las campañas sobre segmentos de alto consumo de datos.

[Volver al Índice](#indice)
